In [1]:
import spacy
import pandas as pd
from collections import Counter
import string

In [4]:

# Загрузка модели Spacy для английского языка
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Удаление знаков препинания
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Приведение текста к нижнему регистру
    text = text.lower()
    return text

def count_word_frequency(tokens):
    # Подсчет частоты каждого слова
    word_freq = Counter(tokens)
    return word_freq


def process_file(file_path):
    # Чтение файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Предобработка текста
    processed_text = preprocess_text(text)

    # Обработка текста с помощью Spacy и токенизация
    doc = nlp(processed_text)
    tokens = [token.text for token in doc if not token.is_space]

    # Подсчет частоты слов
    word_freq = count_word_frequency(tokens)

    # Сортировка слов по убыванию частоты
    sorted_word_freq = word_freq.most_common()

    return sorted_word_freq


In [6]:

file_path = '../learn_words/subtitles/[English] Jared Kushner_ Israel, Palestine, Hamas, Gaza, Iran, and the Middle East _ Lex Fridman Podcast #399 [DownSub.com].txt'  # Замените на путь к вашему файлу
word_freq = process_file(file_path)


In [ ]:
# Структура DataFrame с добавлением столбца 'CommonWord'
data = {
    "Word": [],                  # Слово
    "Frequency": [],             # Частота встречаемости
    "CommonWord": [],            # Флаг для обозначения распространенных слов
    "KnowledgeLevel": [],        # Уровень знания (от 0 до 1)
    "CorrectAnswers": [],        # Счетчик правильных ответов
    "IncorrectAnswers": [],      # Счетчик неправильных ответов
    "HintsUsed": [],             # Счетчик использованных подсказок
    "LastSessionDate": []        # Дата последней сессии
}

df = pd.DataFrame(data)

# Сохранение DataFrame в CSV-файл
df.to_csv("vocabulary_data.csv", index=False)
print(word_freq)

In [10]:
def backup_data(df, backup_filename):
    df.to_csv(backup_filename, index=False)

def select_words_for_session(df, n_words=10):
    # Сортировка слов по частоте и фильтрация топ-100
    top_words = df.sort_values(by='Frequency', ascending=False).head(100)

    # Взвешенный выбор слов на основе уровня знания
    weights = top_words['KnowledgeLevel'].apply(lambda x: 0.1 if x > 0.9 else (0.2 if x > 0.7 else 1)).tolist()
    session_words = top_words.sample(n=n_words, weights=weights)

    return session_words

def update_knowledge_level(df, word, is_correct_answer):
    # Найти строку с соответствующим словом
    word_row = df.loc[df['Word'] == word]

    # Проверка наличия слова в DataFrame
    if word_row.empty:
        print(f"Слово '{word}' не найдено в таблице.")
        return

    # Получение текущих значений
    current_level = word_row['KnowledgeLevel'].iloc[0]
    incorrect_streak = word_row['IncorrectAnswers'].iloc[0]

    # Обновление метрики знания
    if is_correct_answer:
        new_level = min(current_level + 0.1, 1.0)
        incorrect_streak = 0
    else:
        if incorrect_streak == 0:
            new_level = max(current_level - 0.1, 0.0)
        else:
            new_level = max(current_level - 0.3, 0.0)
        incorrect_streak += 1

    # Обновление данных в DataFrame
    df.loc[df['Word'] == word, 'KnowledgeLevel'] = new_level
    df.loc[df['Word'] == word, 'IncorrectAnswers'] = incorrect_streak

In [9]:
print(len(word_freq))

3852
